In [1]:
import pandas as pd

# Load data from S3
users_df = pd.read_csv('s3://277-project-dataset/users.csv')
listings_df = pd.read_csv('s3://277-project-dataset/listings.csv')
seeker_preferences_df = pd.read_csv('s3://277-project-dataset/seeker_preferences.csv')
matches_df = pd.read_csv('s3://277-project-dataset/matches.csv')

# Preprocess data as required
# (e.g., handle missing values, encode categorical variables)


/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/fsspec/registry.py:275: UserWarning: Your installed version of s3fs is very old and known to cause
severe performance issues, see also https://github.com/dask/dask/issues/10276

To fix, you should specify a lower version bound on s3fs, or
update the current installation.

  warnings.warn(s3_msg)


In [105]:
# Inspect dataframes
print("Users dataframe:")
print(users.head())
print("\nSeeker preferences dataframe:")
print(seeker_preferences.head())
print("\nListings dataframe:")
print(listings.head())
print("\nMatches dataframe:")
print(matches.head())

# Check for missing values
print("\nMissing values in users dataframe:")
print(users.isnull().sum())
print("\nMissing values in seeker preferences dataframe:")
print(seeker_preferences.isnull().sum())
print("\nMissing values in listings dataframe:")
print(listings.isnull().sum())
print("\nMissing values in matches dataframe:")
print(matches.isnull().sum())


Users dataframe:
   user_id              name                        email  phone_number  \
0        1   Jeanette Wilson       taylorrice@example.net    7910318354   
1        2    Barbara Peters         robert78@example.org    8068694742   
2        3  Mrs. Lori Hooper  christinarogers@example.com    1202747850   
3        4    Jessica Dunlap     nicoleturner@example.net    1690877939   
4        5        Todd Smith     jonescharles@example.net    9939109444   

  user_type                                      password_hash  
0    seeker  $2b$12$Qri5I4KQV11F2bSqc7yli.vn1pf5O.xIobBxC6J...  
1    seeker  $2b$12$Qri5I4KQV11F2bSqc7yli.vn1pf5O.xIobBxC6J...  
2    seeker  $2b$12$Qri5I4KQV11F2bSqc7yli.vn1pf5O.xIobBxC6J...  
3    seeker  $2b$12$Qri5I4KQV11F2bSqc7yli.vn1pf5O.xIobBxC6J...  
4    lister  $2b$12$Qri5I4KQV11F2bSqc7yli.vn1pf5O.xIobBxC6J...  

Seeker preferences dataframe:
   preference_id  user_id accommodation_type  no_of_people_sharing veg_status  \
0              1        1     

In [15]:
print("\nMissing values in users dataframe:")
print(users.isnull().sum())
print("\nMissing values in seeker preferences dataframe:")
print(seeker_preferences.isnull().sum())
print("\nMissing values in listings dataframe:")
print(listings.isnull().sum())
print("\nMissing values in matches dataframe:")
print(matches.isnull().sum())


Missing values in users dataframe:
user_id          0
name             0
email            0
phone_number     0
user_type        0
password_hash    0
dtype: int64

Missing values in seeker preferences dataframe:
preference_id             0
user_id                   0
accommodation_type        0
no_of_people_sharing    246
veg_status                0
start_date                0
max_budget                0
landmark_preferences    159
no_of_bedrooms            0
no_of_bathrooms           0
apartment_preference    149
city_preference           0
smoking_preference        0
drinking_preference       0
smoker                    0
drinker                   0
dtype: int64

Missing values in listings dataframe:
listing_id                0
user_id                   0
apartment_name            0
address                   0
area                      0
description               0
photos                    0
no_of_bedrooms            0
no_of_bathrooms           0
accommodation_type        0
no_of_pe

In [2]:
# Check the shapes of the DataFrames after preprocessing
print("Users shape:", users_df.shape)
print("Seeker Preferences shape:", seeker_preferences_df.shape)
print("Listings shape:", listings_df.shape)
print("Matches shape:", matches_df.shape)

# Replace empty values in 'no_of_people_sharing' column with 0
seeker_preferences_df['no_of_people_sharing'] = seeker_preferences_df['no_of_people_sharing'].fillna(0)
listings_df['no_of_people_sharing'] = listings_df['no_of_people_sharing'].fillna(0)

# Remove rows with null values in specific columns
cols_to_replace_seekers = ['landmark_preferences', 'apartment_preference']
seeker_preferences_df[cols_to_replace_seekers] = seeker_preferences_df[cols_to_replace_seekers].fillna('no preference')

cols_to_replace_listings = ['description', 'landmark']
listings_df[cols_to_replace_listings] = listings_df[cols_to_replace_listings].fillna('no preference')

seeker_preferences_df.dropna(inplace=True)
listings_df.dropna(inplace=True)

# Remove corresponding rows from matches table
matches_df = matches_df[matches_df['seeker_id'].isin(seeker_preferences_df['user_id'])]
matches_df = matches_df[matches_df['listing_id'].isin(listings_df['listing_id'])]

# Check the shapes of the DataFrames after preprocessing
print("Users shape:", users_df.shape)
print("Seeker Preferences shape:", seeker_preferences_df.shape)
print("Listings shape:", listings_df.shape)
print("Matches shape:", matches_df.shape)

Users shape: (1000, 6)
Seeker Preferences shape: (480, 16)
Listings shape: (520, 21)
Matches shape: (2153, 6)
Users shape: (1000, 6)
Seeker Preferences shape: (480, 16)
Listings shape: (520, 21)
Matches shape: (2153, 6)


In [3]:
# One-hot encode categorical variables with multiple categories
seeker_preferences_df = pd.get_dummies(seeker_preferences_df, columns=['accommodation_type', 'veg_status', 'smoking_preference', 'drinking_preference', 'city_preference'])
listings_df = pd.get_dummies(listings_df, columns=['accommodation_type', 'veg_status', 'has_smoker', 'has_drinker', 'city'])

# Encode 'user_type' column in users_df
users_df['user_type'] = users_df['user_type'].map({'seeker': 0, 'lister': 1})

In [4]:
# Merge matches with seeker_preferences and listings
matches_merged = matches_df.merge(seeker_preferences_df, left_on='seeker_id', right_on='user_id', how='left')
matches_merged = matches_merged.merge(listings_df, on='listing_id', how='left')

# Drop unnecessary columns
matches_merged.drop(['user_id_y'], axis=1, inplace=True)
matches_merged.rename(columns={'user_id_x': 'seeker_id'}, inplace=True)

# Display the merged data
matches_merged.head()

,match_id,seeker_id,listing_id,status,generated_datetime,updated_datetime,preference_id,seeker_id,no_of_people_sharing_x,start_date,...,accommodation_type_single_y,veg_status_any_y,veg_status_non-veg_y,veg_status_veg_y,has_smoker_no,has_smoker_yes,has_drinker_no,has_drinker_yes,city_San Francisco,city_San Jose
0,1,1,441,accepted,2023-12-15 11:41:08.969050,2024-03-04 14:03:34.071672,1,1,0.0,2024-02-25,...,True,False,True,False,False,True,False,True,False,True
1,2,1,32,no_action,2023-09-13 23:40:37.287733,2023-09-13 23:40:37.287733,1,1,0.0,2024-02-25,...,False,True,False,False,True,False,False,True,False,True
2,3,1,101,no_action,2024-01-26 18:16:09.852571,2024-01-26 18:16:09.852571,1,1,0.0,2024-02-25,...,True,True,False,False,False,True,False,True,False,True
3,4,1,114,no_action,2023-09-06 06:08:57.778300,2023-09-06 06:08:57.778300,1,1,0.0,2024-02-25,...,False,True,False,False,False,True,False,True,False,True
4,5,1,125,no_action,2023-09-20 00:13:15.348585,2023-09-20 00:13:15.348585,1,1,0.0,2024-02-25,...,True,True,False,False,True,False,False,True,False,True


In [5]:
#Remove the seeker id duplicate column
matches_merged = matches_merged.loc[:,~matches_merged.columns.duplicated()].copy()

In [8]:
# Split the data into train and test sets
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(matches_merged, test_size=0.2, random_state=42)

In [9]:
from sklearn.preprocessing import LabelEncoder

# Initialize the LabelEncoder
status_encoder = LabelEncoder()

# Encode the 'status' column
train_data['status'] = status_encoder.fit_transform(train_data['status'])


In [11]:
from surprise import Dataset, Reader
# Load the training data into Surprise's Dataset object
reader = Reader(rating_scale=(0, 2))  # Ratings: 0 = Declined, 1 = No action, 2 = Accepted
train_dataset = Dataset.load_from_df(train_data[['seeker_id', 'listing_id', 'status']], reader)

# Build the trainset
trainset = train_dataset.build_full_trainset()

In [13]:
from surprise import SVD, Dataset, Reader
from surprise.model_selection import GridSearchCV
param_grid = {'n_epochs': [5, 10], 'lr_all': [0.002, 0.005]}
gs = GridSearchCV(SVD, param_grid, measures=['rmse', 'mae'], cv=3)
gs.fit(train_dataset)

In [14]:
# Get the best parameters
best_params = gs.best_params['rmse']

# Train the model with the best parameters
model = SVD(n_epochs=best_params['n_epochs'], lr_all=best_params['lr_all'])
model.fit(trainset)

In [15]:
# Test the model
testset = trainset.build_testset()
predictions = model.test(testset)

# Evaluate the model
from surprise import accuracy
accuracy.rmse(predictions)
accuracy

RMSE: 0.6079


<module 'surprise.accuracy' from '/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/surprise/accuracy.py'>

In [17]:
import numpy as np
# Define a new row based on seeker_preferences.csv schema with appropriate columns
new_seeker_data = {
    'preference_id': 481,
    'user_id': 1001,
    'accommodation_type': 'single',
    'no_of_people_sharing': np.nan,
    'veg_status': 'veg',
    'start_date': '2024-05-05',
    'max_budget': 1500,
    'landmark_preferences': np.nan,
    'no_of_bedrooms': 1,
    'no_of_bathrooms': 1,
    'apartment_preference': '1BHK',
    'city_preference': 'San Francisco',
    'smoking_preference': 'no',
    'drinking_preference': 'no',
    'smoker': 'no',
    'drinker': 'no'
}

# Create a DataFrame from the dictionary
new_seeker_df = pd.DataFrame([new_seeker_data])

# Replace empty values in 'no_of_people_sharing' column with 0
new_seeker_df['no_of_people_sharing'] = new_seeker_df['no_of_people_sharing'].fillna(0)

# Remove rows with null values in specific columns
cols_to_replace_seekers = ['landmark_preferences', 'apartment_preference']
new_seeker_df[cols_to_replace_seekers] = new_seeker_df[cols_to_replace_seekers].fillna('no preference')


# Align with the original DataFrame's columns
new_seeker_df = new_seeker_df.reindex(columns=seeker_preferences_df.columns)

# Preprocess: one-hot encoding with accurate column names from seeker_preferences_df
categorical_columns = ['accommodation_type', 'veg_status', 'smoking_preference', 'drinking_preference', 'city_preference']
categorical_columns = [col for col in categorical_columns if col in new_seeker_df.columns]

new_seeker_df = pd.get_dummies(new_seeker_df, columns=categorical_columns)

# Align with the training DataFrame
aligned_sample = pd.DataFrame(columns=seeker_preferences_df.columns)
aligned_sample.update(new_seeker_df)
aligned_sample = aligned_sample.fillna(0)

# Generate top 5 recommendations
def get_top_5_recommendations(seeker_id, model, listings_df):
    listing_scores = {}
    for listing_id in listings_df['listing_id']:
        score = model.predict(uid=seeker_id, iid=listing_id).est
        listing_scores[listing_id] = score
    top_5_listings = sorted(listing_scores.items(), key=lambda x: x[1], reverse=True)[:5]
    return listings_df[listings_df['listing_id'].isin([x[0] for x in top_5_listings])]

# Sample seeker ID (based on the new user ID)
seeker_id = new_seeker_data['user_id']

# Uncomment the following line and run it once your model is available and trained:
recommendations = get_top_5_recommendations(seeker_id=seeker_id, model=model, listings_df=listings_df)
recommendations


,listing_id,user_id,apartment_name,address,area,description,photos,no_of_bedrooms,no_of_bathrooms,no_of_people_sharing,...,accommodation_type_single,veg_status_any,veg_status_non-veg,veg_status_veg,has_smoker_no,has_smoker_yes,has_drinker_no,has_drinker_yes,city_San Francisco,city_San Jose
5,6,12,Park Path,Suite 535,1074,modern flat,agent.jpg,3,2,3.0,...,False,True,False,False,False,True,True,False,False,True
16,17,36,Angela Extensions,Apt. 504,1509,cozy room,beautiful.jpg,3,1,0.0,...,True,True,False,False,False,True,False,True,False,True
31,32,66,Delgado Islands,Suite 769,2349,elegant house,wide.jpg,3,1,3.0,...,False,True,False,False,True,False,False,True,False,True
79,80,146,Mccormick Locks,Apt. 811,2526,elegant house,risk.jpg,3,1,0.0,...,True,True,False,False,False,True,False,True,True,False
95,96,174,Anthony Squares,Suite 499,2528,modern flat,those.jpg,3,1,0.0,...,True,True,False,False,True,False,True,False,True,False


In [170]:
import pickle

# Save the model to a pickle file
with open('svd_model.pkl', 'wb') as file:
    pickle.dump(model, file)


In [28]:
import boto3

# Initialize the S3 client
s3 = boto3.client('s3')

# Specify the bucket name
bucket_name = '277-new-model'

# Upload the pickle file
s3.upload_file('svd_model.pkl', bucket_name, 'models/svd_model.pkl')

# Upload the inference script
s3.upload_file('interface.py', bucket_name, 'models/interface.py')

# Upload listings
s3.upload_file('listings.csv', bucket_name, 'models/listings.csv')

# Upload the tarball
s3.upload_file('svd_model.tar.gz', bucket_name, 'models/svd_model.tar.gz')

In [23]:
import boto3

# Initialize the S3 client
s3 = boto3.client('s3')
bucket_name = '277-new-model'
# Upload the tarball
s3.upload_file('svd_model.tar.gz', bucket_name, 'svd_model.tar.gz')

In [27]:
import sagemaker
from sagemaker.sklearn import SKLearnModel
from sagemaker import get_execution_role

# Get SageMaker execution role
role = get_execution_role()

# S3 path to the .tar.gz file containing the model and inference script
model_uri = 's3://277-new-model/svd_model.tar.gz'

# Deploy the model using the SageMaker SKLearnModel
model = SKLearnModel(
    model_data=model_uri,
    role=role,
    entry_point='interface-new.py',  # This refers to the script inside the .tar.gz
    framework_version='0.23-1',  # Choose the appropriate Scikit-Learn version
    py_version='py3'             # Choose the appropriate Python version
)

# Deploy the model
predictor = model.deploy(
    initial_instance_count=1,
    instance_type='ml.t2.medium',  # Adjust based on your performance needs
    endpoint_name='svd-recommendation-new-2'
)


-------------------------------------------

KeyboardInterrupt: 